# Imports

In [1]:
import os
import json
import random
import subprocess
from collections import defaultdict

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
import optuna
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from utils.Loader import CardsDataset
from arquitecture.CardsClassifier import CardClassifier
from utils.AttentionGradCam import AttentionGradCAM

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Currently using", DEVICE)

Currently using cuda


# Hyperparameters

In [2]:
EPOCH = 300
BATCH_SIZE = 2400
NUN_WORKERS = 6
LR = 0.0004
N_TRIALS = 100
SEED = 55

# Load the pretrained models

In [3]:
csv_file = "cards.csv"
target = "suit"

suit_train_dataset = CardsDataset(scale=0.6, split="train", csv_file=csv_file, target=target)
suit_test_dataset = CardsDataset(scale=0.6, split="test", csv_file=csv_file, target=target)
suit_valid_dataset = CardsDataset(scale=0.6, split="valid", csv_file=csv_file, target=target)

suit_train_loader = DataLoader(suit_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)
suit_valid_loader = DataLoader(suit_valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)
suit_test_loader  = DataLoader(suit_test_dataset,  batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)

_, label = suit_test_dataset.__getitem__(1)

suit_classifier = CardClassifier(image_size=torch.Size((134, 134)), 
                            convolution_structure=[1,8,8,16,16,24,24,32,32],
                            expert_output_len=1,
                            expert_depth=4,
                            output_len=len(label),
                            pool_depth=2
                            ).to(DEVICE)

print(suit_classifier.n_parameters())
print(len(label))

2373308
5


In [4]:
csv_file = "cards.csv"
target = "category"

category_train_dataset = CardsDataset(scale=0.6, split="train", csv_file=csv_file, target=target)
category_test_dataset = CardsDataset(scale=0.6, split="test", csv_file=csv_file, target=target)
category_valid_dataset = CardsDataset(scale=0.6, split="valid", csv_file=csv_file, target=target)

category_train_loader = DataLoader(category_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)
category_valid_loader = DataLoader(category_valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)
category_test_loader  = DataLoader(category_test_dataset,  batch_size=BATCH_SIZE, shuffle=True, num_workers=NUN_WORKERS)

_, label = category_test_dataset.__getitem__(1)

category_classifier = CardClassifier(image_size=torch.Size((134, 134)), 
                            convolution_structure=[1,8,8,16,16,24,24,32,32],
                            expert_output_len=1,
                            expert_depth=4,
                            output_len=len(label),
                            pool_depth=2
                            ).to(DEVICE)

print(len(label))

14


In [5]:
print(suit_classifier.n_parameters())
print(category_classifier.n_parameters())

2373308
2373449


In [6]:

category_checkpoint = torch.load("models/checkpoints/category_classifier_checkpoint.pth")
category_classifier.load_state_dict(category_checkpoint['model_state_dict'])
category_classifier.eval()
category_classifier.to(DEVICE)

suit_checkpoint = torch.load("models/checkpoints/suit_classifier_checkpoint.pth")
suit_classifier.load_state_dict(suit_checkpoint['model_state_dict'])
suit_classifier.eval()
suit_classifier.to(DEVICE)

print("Loaded models")

Loaded models


# GradCam

In [7]:
def generateGradCam(result_path: str, split: str, sample_size: int, model: nn.Module, dataset: Dataset):
    target_layer = model.cnn_block.layers[-1]
    attention_layer = model.attention_block.batch_norm
    grad_cam = AttentionGradCAM(model, target_layer, attention_layer)

    # Dict to organize the dictionaries
    indices_por_label = defaultdict(list)
    for idx in range(len(dataset)):
        _, label = dataset[idx]
        if hasattr(label, 'argmax'):
            label_int = label.argmax(dim=0).item()
        else:
            label_int = label
        indices_por_label[label_int].append(idx)

    # For each categorie get at least a sample of the sample_size
    for label_val, indices in indices_por_label.items():
        if len(indices) > sample_size:
            sample_indices = random.sample(indices, sample_size)
        else:
            sample_indices = indices

        # Create subplots
        fig, axes = plt.subplots(nrows=len(sample_indices), ncols=2, figsize=(10, 2 * len(sample_indices)))
        fig.suptitle(f"Category {(label_val)}", fontsize=16)

        # Just if the there is only one subplot
        if len(sample_indices) == 1:
            axes = np.expand_dims(axes, axis=0)

        for i, idx in enumerate(sample_indices):
            img, _ = dataset[idx]
            input_tensor = img.unsqueeze(0).to(DEVICE)
            cam, predicted_class = grad_cam.generate_cam(input_tensor)

            # Resize the heatmap into the image size
            cam = cv2.resize(cam, (img.shape[-1], img.shape[-2]))

            # Parse the heatmap into rgb
            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

            # Parse the image into rgb instead of grey scale
            img_np = img.squeeze().cpu().numpy()
            img_np = cv2.cvtColor(np.uint8(255 * img_np), cv2.COLOR_GRAY2RGB)

            # Overlab the heatmap
            superimposed_img = cv2.addWeighted(heatmap, 0.4, img_np, 0.6, 0)

            # Show the original image
            axes[i, 0].imshow(img_np)
            axes[i, 0].set_title(f"Original (class {(label_val)})")
            axes[i, 0].axis("off")

            # Show the gradcam 
            axes[i, 1].imshow(superimposed_img)
            axes[i, 1].set_title(f"Grad-CAM (prediction {(predicted_class)})")
            axes[i, 1].axis("off")

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        os.makedirs(f"{result_path}/{split}/GradCam/", exist_ok=True)
        plt.savefig(f"{result_path}/{split}/GradCam/GradCam_{(label_val)}.png")
        
        plt.close()

In [8]:
generateGradCam("result", "category", 5, category_classifier, category_test_dataset)
generateGradCam("result", "suit", 5, suit_classifier, suit_test_dataset)

/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/torch/nn/modules/module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


# Tree Representation

In [9]:
def prediction(image, model):
    output = model(image.to("cuda"))
    pred_class = torch.argmax(output, dim=1)
    return pred_class

def get_results(test, model: nn.Module):
    true_labels = []
    pred_labels = []
    expert_outputs = {}
    model.eval()
    test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)
    with torch.no_grad():
        for images, labels in test_loader:
            predictions = prediction(images, model)

            for pred, label in zip(predictions, labels):
                label_idx = torch.argmax(label).item()
                true_labels.append(label_idx)
                pred_labels.append(pred.item())
                
            new_outputs = model.get_expert_output_dict()
            if not expert_outputs:
                expert_outputs = new_outputs
            else:
                for k in new_outputs:
                    expert_outputs[k] += new_outputs[k]
    
    expert_outputs["true_label"] = true_labels
    expert_outputs["pred_label"] = pred_labels
    
    return true_labels, pred_labels, expert_outputs

def get_results_expert(test_dataset, valid_dataset, model: nn.Module):
    _, _, test_expert_dataset = get_results(test=test_dataset, model=model)
    df_test = pd.DataFrame(test_expert_dataset)
    _,_ , valid_expert_dataset = get_results(test=valid_dataset, model=model)
    df_valid = pd.DataFrame(valid_expert_dataset)
    df_combined = pd.concat([df_test, df_valid], ignore_index=True)
    return df_combined

In [10]:
df_suit = get_results_expert(test_dataset=suit_test_dataset, valid_dataset=suit_valid_dataset, model=suit_classifier)
df_category = get_results_expert(test_dataset=category_test_dataset, valid_dataset=category_valid_dataset, model=category_classifier)

In [11]:
df_suit.to_csv("result/suit/suit_expert.csv", index=False)
df_category.to_csv("result/category/category_expert.csv", index=False)

# Trees

In [12]:

def tree_objective(trial, X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    max_depth = trial.suggest_int("max_depth", 1, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    clf = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = precision_score(y_test, y_pred, average='macro', zero_division=0)
    return score

def find_best_tree_params_optuna(X, Y, n_trials=50):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: tree_objective(trial, X, Y), n_trials=n_trials, n_jobs=NUN_WORKERS)
    print("Best Macro Precision Score:", study.best_trial.value)
    return study.best_trial
   
def get_decision_tree_svg(csv_file:str, result_path: str, target_label: bool):
    df = pd.read_csv(csv_file)
    y = df[target_label]
    X = df.drop(columns=["true_label", "pred_label"]).loc[:, (df != 0).any(axis=0)]
    
    best_best_trial = find_best_tree_params_optuna(X, y, n_trials=N_TRIALS)

    os.makedirs(result_path, exist_ok=True)

    with open(f"{result_path}/best_params.json", "w") as f:
        temp_dict = {"Best_params": best_best_trial.params, "Precision_Score": best_best_trial.value}
        json.dump(temp_dict, f, indent=4)

    clf = DecisionTreeClassifier(**best_best_trial.params, random_state=SEED)
    clf.fit(X, y)

    dot_file = f"{result_path}/tree.dot"
    
    with open(dot_file, "w") as f:
        export_graphviz(clf,
                        out_file=f,
                        feature_names=X.columns,
                        class_names=[str(cls) for cls in clf.classes_],
                        filled=True,
                        rounded=True,
                        special_characters=True)

    svg_file = f"{result_path}/{target_label}.svg"
    subprocess.run(["dot", "-Tsvg", dot_file, "-o", svg_file], check=True)


In [14]:
get_decision_tree_svg(csv_file="result/suit/suit_expert.csv", result_path="result", target_label="pred_label")
get_decision_tree_svg(csv_file="result/suit/suit_expert.csv", result_path="result", target_label="true_label")
get_decision_tree_svg(csv_file="result/category/category_expert.csv", result_path="result", target_label="pred_label")
get_decision_tree_svg(csv_file="result/category/category_expert.csv", result_path="result", target_label="true_label")

[I 2025-05-07 15:16:04,633] A new study created in memory with name: no-name-fc5c2552-d194-4d5f-bbed-1446a82ddc5a


[W 2025-05-07 15:16:04,649] Trial 1 failed with parameters: {'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 2, 'criterion': 'gini'} because of the following error: ValueError('at least one array or dtype is required').
Traceback (most recent call last):
  File "/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_42134/559766688.py", line 21, in <lambda>
    study.optimize(lambda trial: tree_objective(trial, X, Y), n_trials=n_trials, n_jobs=NUN_WORKERS)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_42134/559766688.py", line 14, in tree_objective
    clf.fit(X_train, y_train)
  File "/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: at least one array or dtype is required